In [1]:
# pacotes

import requests
import pandas as pd
import json as json
import psycopg2
from sqlalchemy import create_engine, MetaData
from sqlalchemy.dialects.postgresql import insert
from urllib.parse import quote_plus
from dotenv import load_dotenv
import os

In [ ]:
# Caminho para o .env da EUROLASER (Railway)

# Depois essa pasta vamos trocar pela pasta que ficará no servidor da EUROLASER
dotenv_path = "URL"

# Carrega as variáveis do arquivo .env, forçando substituição se já houver algo na memória
load_dotenv(dotenv_path, override=True)

# Recupera variáveis de ambiente
host = os.getenv("host")
porta = os.getenv("porta")
usuario = os.getenv("usuario")
senha = quote_plus(os.getenv("senha"))  # Protege caracteres especiais
banco = os.getenv("database")

# Mostra para confirmação
print("Conectando em:", host, porta, usuario, banco)

# Cria a engine
engine = create_engine(f"postgresql://{usuario}:{senha}@{host}:{porta}/{banco}")

Conectando em: metro.proxy.rlwy.net 14053 postgres railway


In [ ]:
# Carrega o arquivo com as credenciais

with open('CREDENCIAIS') as f:
    config = json.load(f)

token = config['TOKEN_API']

In [ ]:
# API OP

pagina_op = 1
todos_dados_op = []

# Loop de paginação
while True:
    url_op = f"API_OP"
    headers_op = {
        "Content-Type": "application/json",
        "Token": token
    }

    response = requests.get(url_op, headers=headers_op)
    
    if response.status_code == 200:
        dados = response.json()
        
        # Se a resposta vier vazia, terminamos a coleta
        if not dados:
            break

        todos_dados_op.extend(dados)
        print(f"✅ Página {pagina_op} carregada. Total acumulado: {len(todos_dados_op)}")
        pagina_op += 1
    else:
        print(f"❌ Erro na página {pagina_op}: {response.status_code} - {response.text}")
        break

# Converter em DataFrame
df_ops = pd.DataFrame(todos_dados_op)

# Visualizar
print("🔍 Primeiras ordens de produção:")
print(df_ops.head())

✅ Página 1 carregada. Total acumulado: 300
✅ Página 2 carregada. Total acumulado: 600
✅ Página 3 carregada. Total acumulado: 900
✅ Página 4 carregada. Total acumulado: 1200
✅ Página 5 carregada. Total acumulado: 1500
✅ Página 6 carregada. Total acumulado: 1800
✅ Página 7 carregada. Total acumulado: 2100
✅ Página 8 carregada. Total acumulado: 2400
✅ Página 9 carregada. Total acumulado: 2700
✅ Página 10 carregada. Total acumulado: 3000
✅ Página 11 carregada. Total acumulado: 3300
✅ Página 12 carregada. Total acumulado: 3600
✅ Página 13 carregada. Total acumulado: 3900
✅ Página 14 carregada. Total acumulado: 4049
❌ Erro na página 15: 400 - [{"method-error-400":"N\u00E3o foi poss\u00EDvel buscar OP: Nenhum registro encontrado."}]
🔍 Primeiras ordens de produção:
   numero parte    codigo   dt_inicio periodo pedido tipo  impof codcli  \
0  044494    00      S001  08/01/2025    0001  28281           0  03324   
1  044495    00      S001  08/01/2025    0001  28281           0  03324   
2  0444

In [ ]:
# API PEDIDOS

pagina_pedidos = 1
todos_dados_pedidos = []
limite_paginas = 1000000  

while True:
    url_pedidos = f"API/PEDIDOS"

    headers_pedidos = {
        "Content-Type": "application/json",
        "Token": token
    }

    response = requests.get(url_pedidos, headers=headers_pedidos)
    
    if response.status_code == 200:
        dados_pedidos = response.json()
        
        if not dados_pedidos:
            break

        todos_dados_pedidos.extend(dados_pedidos)
        print(f"✅ Página {pagina_pedidos} carregada. Total acumulado: {len(todos_dados_pedidos)}")

        pagina_pedidos += 1

        # Interrompe
        if pagina_pedidos > limite_paginas:
            print("🚧 Limite de páginas atingido (teste)")
            break

    else:
        print(f"❌ Erro na página {pagina_pedidos}: {response.status_code} - {response.text}")
        break

# Converte em DataFrame
df_pedidos = pd.DataFrame(todos_dados_pedidos)
print(df_pedidos.head())


✅ Página 1 carregada. Total acumulado: 300
✅ Página 2 carregada. Total acumulado: 600
✅ Página 3 carregada. Total acumulado: 900
✅ Página 4 carregada. Total acumulado: 1200
✅ Página 5 carregada. Total acumulado: 1500
✅ Página 6 carregada. Total acumulado: 1800
✅ Página 7 carregada. Total acumulado: 1870
❌ Erro na página 8: 400 - [{"method-error-400":"N\u00E3o foi poss\u00EDvel buscar pedido: Nenhum registro encontrado."}]
  numero     ped_cli  dt_emissao codcli  \
0  28283              08/01/2025  03150   
1  28286       84141  08/01/2025  00057   
2  28287       84138  08/01/2025  00057   
3  28288       84132  08/01/2025  00057   
4  28289  4500363920  08/01/2025  02175   

                                               nome                cnpj  \
0             OCS CONFECCOES, ATACADO E VAREJO LTDA  30.970.168/0001-90   
1  WARUSKY COMERCIO INDUSTRIA E REPRESENTACOES LTDA  79.674.297/0001-20   
2  WARUSKY COMERCIO INDUSTRIA E REPRESENTACOES LTDA  79.674.297/0001-20   
3  WARUSKY COME

In [ ]:
# Consumindo a API de Clientes

pagina = 1
todos_dados = []

# Loop de paginação
while True:
    url = f"API/CLIENTES"
    headers = {
        "Content-Type": "application/json",
        "Token": token
    }

    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        dados = response.json()
        
        # Se a resposta vier vazia, terminamos a coleta
        if not dados:
            break

        todos_dados.extend(dados)
        print(f"✅ Página {pagina} carregada. Total acumulado: {len(todos_dados)}")
        pagina += 1
    else:
        print(f"❌ Erro na página {pagina}: {response.status_code} - {response.text}")
        break

# Converter em DataFrame
df_clientes = pd.DataFrame(todos_dados)

# Visualizar
print("🔍 Primeiras ordens de produção:")
print(df_clientes.head())

✅ Página 1 carregada. Total acumulado: 300
✅ Página 2 carregada. Total acumulado: 486
❌ Erro na página 3: 400 - [{"method-error-400":"N\u00E3o foi poss\u00EDvel buscar entidade: Nenhum registro encontrado."}]
🔍 Primeiras ordens de produção:
                                 nome                            fantasia  \
0                    KMNO SPORTS LTDA                    KMNO SPORTS LTDA   
1                    GABRIELA PACHECO                    GABRIELA PACHECO   
2                       AMANDA MAYUMI                       AMANDA MAYUMI   
3  HELEN TICIANE BARROS DE ANUNCIAÇÃO  HELEN TICIANE BARROS DE ANUNCIAÇÃO   
4                   JESSICA DAMILELLI                   JESSICA DAMINELLI   

       telefone telefone2                         email  \
0  47 9696-5303                    nfe@eurolaser.com.br   
1   54981236444            gabrielasantiramos@gmail.com   
2   11985102919                    amm.mayumi@gmail.com   
3   54981438438                     helbarros@gmail.com   
4

In [7]:
# verificando registros 

df_clientes.head()

,nome,fantasia,telefone,telefone2,email,endereco,bairro,cep,nome_cid,cod_est,...,nome_cid_cob,cod_est_cob,sit_dup,historico,dt_nasc,cod_atvd,codrep,inadimplente,contatos,classificacao
0,KMNO SPORTS LTDA,KMNO SPORTS LTDA,47 9696-5303,,nfe@eurolaser.com.br,RUA 3030,CENTRO,88330311,BALNEARIO CAMBORIU,SC,...,BALNEARIO CAMBORIU,SC,,9001,,001,[],N,[],[]
1,GABRIELA PACHECO,GABRIELA PACHECO,54981236444,,gabrielasantiramos@gmail.com,RUA SAO MATEUS,SÃO MATEUS,36025001,JUIZ DE FORA,MG,...,JUIZ DE FORA,MG,,000,,001,[0008],N,[],[]
2,AMANDA MAYUMI,AMANDA MAYUMI,11985102919,,amm.mayumi@gmail.com,ALAMEDA JAÚ,JARDIM PAULISTA,01420002,SAO PAULO,SP,...,SAO PAULO,SP,00,000,,001,[0008],N,[],[]
3,HELEN TICIANE BARROS DE ANUNCIAÇÃO,HELEN TICIANE BARROS DE ANUNCIAÇÃO,54981438438,,helbarros@gmail.com,PAUL HARRIS,CENTRO,95680000,CANELA,RS,...,CANELA,RS,,000,,001,[0008],N,[],[]
4,JESSICA DAMILELLI,JESSICA DAMINELLI,99908-6392,,jessidaminelli@gmail.com,HELVÉCIO COELHO RODRIGUES,SANTA BARBARA,88802040,CRICIUMA,SC,...,CRICIUMA,SC,,,,,[0008],N,[],[]


In [8]:
# seleciona as tabelas importantes

df_clientes  = df_clientes [['nome','fantasia','telefone','email','nome_cid','cod_est','ativo','codcli']]
df_clientes.head()

,nome,fantasia,telefone,email,nome_cid,cod_est,ativo,codcli
0,KMNO SPORTS LTDA,KMNO SPORTS LTDA,47 9696-5303,nfe@eurolaser.com.br,BALNEARIO CAMBORIU,SC,S,05922
1,GABRIELA PACHECO,GABRIELA PACHECO,54981236444,gabrielasantiramos@gmail.com,JUIZ DE FORA,MG,S,05923
2,AMANDA MAYUMI,AMANDA MAYUMI,11985102919,amm.mayumi@gmail.com,SAO PAULO,SP,S,05924
3,HELEN TICIANE BARROS DE ANUNCIAÇÃO,HELEN TICIANE BARROS DE ANUNCIAÇÃO,54981438438,helbarros@gmail.com,CANELA,RS,S,05925
4,JESSICA DAMILELLI,JESSICA DAMINELLI,99908-6392,jessidaminelli@gmail.com,CRICIUMA,SC,S,00838


Renomear as colunas que ja existem para nao precisar trocar nas rotas/apis/front etc...

1. id
2. codigo
3. nome
4. nomefantasia

In [9]:
# Rename

df_clientes.rename(columns={
    'codcli': 'codigo',
    'fantasia': 'nomeFantasia'
}, inplace=True)

In [10]:
df_clientes.head()

,nome,nomeFantasia,telefone,email,nome_cid,cod_est,ativo,codigo
0,KMNO SPORTS LTDA,KMNO SPORTS LTDA,47 9696-5303,nfe@eurolaser.com.br,BALNEARIO CAMBORIU,SC,S,05922
1,GABRIELA PACHECO,GABRIELA PACHECO,54981236444,gabrielasantiramos@gmail.com,JUIZ DE FORA,MG,S,05923
2,AMANDA MAYUMI,AMANDA MAYUMI,11985102919,amm.mayumi@gmail.com,SAO PAULO,SP,S,05924
3,HELEN TICIANE BARROS DE ANUNCIAÇÃO,HELEN TICIANE BARROS DE ANUNCIAÇÃO,54981438438,helbarros@gmail.com,CANELA,RS,S,05925
4,JESSICA DAMILELLI,JESSICA DAMINELLI,99908-6392,jessidaminelli@gmail.com,CRICIUMA,SC,S,00838


Precisamos criar um processo, que verifique se todos os clientes na tabela de ordens de produção estão na tabela de clientes, se houverem clientes que não estão, precisamos inserir antes de salvar 

In [11]:
df_pedidos.head(1)

,numero,ped_cli,dt_emissao,codcli,nome,cnpj,codrep,nome_rep,per_desc,vlr_desc,...,valor,faturado,cancelado,sit_dup,colecao,valor_total_liq,valor_total_fat,valor_total_pen,valor_total_bruto,dt_altera
0,28283,,08/01/2025,03150,"OCS CONFECCOES, ATACADO E VAREJO LTDA",30.970.168/0001-90,00000,LABRO,0.0,0.0,...,0.0,6000.0,0.0,11,00,2100.0,2100.0,0.0,2100.0,08/01/2025 11:20:30


In [12]:
# lista de clientes 

lista_clientes = df_clientes['codigo'].tolist()

# Verificando se esses clientes da lista de OP estão no dataframe de OP
df_ops ['flagvalidacao'] = df_ops['codcli'].isin(lista_clientes).astype(int)
df_pedidos ['flagvalidacao'] = df_pedidos ['codcli'].isin(lista_clientes).astype(int)

In [13]:
# verificando

df_ops['flagvalidacao'].value_counts()

flagvalidacao
1    3997
0      52
Name: count, dtype: int64

In [14]:
# verificando

df_pedidos['flagvalidacao'].value_counts()

flagvalidacao
1    1826
0      44
Name: count, dtype: int64

In [15]:
# verificando as ops
df_ops_nao_identificados = df_ops.query("flagvalidacao == 0")
qtd_ordensnao_identificados = df_ops_nao_identificados ['numero'].nunique()
qtd_clientesnao_identificados = df_ops_nao_identificados ['codcli'].nunique()


# verificando os pedidos
df_pedidos_nao_identificados = df_pedidos.query("flagvalidacao == 0")
qtd_pedidosnao_identificados = df_pedidos_nao_identificados ['numero'].nunique()
qtd_clientesnao_identificados_pedidos = df_pedidos_nao_identificados ['codcli'].nunique()


print("Quantidade de ordens com clientes não identificados ", qtd_ordensnao_identificados)
print("Quantidade de clientes não identificados: ", qtd_clientesnao_identificados, "\n")

print("Quantidade de Pedidos com clientes não identificados ", qtd_pedidosnao_identificados)
print("Quantidade de clientes não identificados: ", qtd_clientesnao_identificados_pedidos)

Quantidade de ordens com clientes não identificados  52
Quantidade de clientes não identificados:  13 

Quantidade de Pedidos com clientes não identificados  44
Quantidade de clientes não identificados:  18


In [16]:
# criando uma lista, somente com os clientes que nao estao na tabela de clientes

lista_clientes_nao_identificados = df_ops_nao_identificados ['codcli'].drop_duplicates().tolist()
lista_clientes_nao_identificados_pedidos = df_pedidos_nao_identificados['codcli'].drop_duplicates().tolist()

In [17]:
# exibindo a lista

lista_clientes_nao_identificados

['00646',
 '03479',
 '00346',
 '00233',
 '00494',
 '01230',
 '00439',
 '01071',
 '04008',
 '01170',
 '00644',
 '00205',
 '00635']

In [18]:
# exibindo a lista

lista_clientes_nao_identificados_pedidos

['01868',
 '00646',
 '03479',
 '00346',
 '00233',
 '00501',
 '00494',
 '01230',
 '00439',
 '01071',
 '02547',
 '04008',
 '01170',
 '00644',
 '00138',
 '02135',
 '00635',
 '04587']

In [19]:
# unindo as duas listas

lista_clientes_nao_identificados = list(
    set(lista_clientes_nao_identificados + lista_clientes_nao_identificados_pedidos)
)


In [20]:
df_clientes.head()

,nome,nomeFantasia,telefone,email,nome_cid,cod_est,ativo,codigo
0,KMNO SPORTS LTDA,KMNO SPORTS LTDA,47 9696-5303,nfe@eurolaser.com.br,BALNEARIO CAMBORIU,SC,S,05922
1,GABRIELA PACHECO,GABRIELA PACHECO,54981236444,gabrielasantiramos@gmail.com,JUIZ DE FORA,MG,S,05923
2,AMANDA MAYUMI,AMANDA MAYUMI,11985102919,amm.mayumi@gmail.com,SAO PAULO,SP,S,05924
3,HELEN TICIANE BARROS DE ANUNCIAÇÃO,HELEN TICIANE BARROS DE ANUNCIAÇÃO,54981438438,helbarros@gmail.com,CANELA,RS,S,05925
4,JESSICA DAMILELLI,JESSICA DAMINELLI,99908-6392,jessidaminelli@gmail.com,CRICIUMA,SC,S,00838


In [21]:
# Pega as colunas do DataFrame original

colunas = df_clientes.columns.tolist()

# Cria uma lista de dicionários com os novos clientes

dados_novos_clientes = []
for codcli in lista_clientes_nao_identificados:
    cliente_dict = {col: ("Não Identificado" if col != "codigo" else codcli) for col in colunas}
    dados_novos_clientes.append(cliente_dict)

# Cria o novo DataFrame com os registros ausentes

novos_clientes = pd.DataFrame(dados_novos_clientes)

# Garante a mesma ordem de colunas

novos_clientes = novos_clientes[df_clientes.columns]

# Concatena com o df original

df_clientes = pd.concat([df_clientes, novos_clientes], ignore_index=True)

print(f"✅ Clientes adicionados: {len(novos_clientes)}")

✅ Clientes adicionados: 19


In [22]:
# Reflete a estrutura do banco para capturar a tabela já existen
metadata = MetaData()
metadata.reflect(bind=engine)
tabela_cliente = metadata.tables["Cliente"]  # Tabela com nome e capitalização exatos

# Converte DataFrame em lista de dicionários
dados = df_clientes.to_dict(orient="records")

# Cria comando de insert com fallback se houver conflito de PK
stmt = insert(tabela_cliente).values(dados)
stmt = stmt.on_conflict_do_nothing(index_elements=["codigo"])  # PK = codigo

# Executa a operação no banco
with engine.begin() as conn:
    conn.execute(stmt)
